# Wissensaggregator Mittelalter und frühe Neuzeit

## Daten für Diözesen übertragen

Lies Daten aus `Kloester und Stifte_neu_2020.accdb`.

- [Diözesen](#Diözesen)
  - [Einträge in `item`](#Einträge-in-item)
  - [Externe URLs eintragen](#Externe-URLs-eintragen)
  - [Alternative Namen eintragen](#Alternative-Namen-eintragen)
  - [Bischofssitze zuordnen](#Bischofssitze-zuordnen)
  - [Literaturverweise](#Literaturverweise)

Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise

In [6]:
using WiagDataSetup

In [7]:
using MySQL, CSV, DataFrames

In [9]:
includet("src/Bistum.jl")

In [10]:
Wds = WiagDataSetup

WiagDataSetup

In [11]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

Trage die jeweils aktuelle Datenquelle ein!

In [12]:
data_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bistümer-2022-11-10"

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bistümer-2022-11-10"

In [17]:
item_type_id = 1

1

In [18]:
check_globals()

┌ Info: WiagDataSetup
└ @ Main C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\Bistum.jl:13
┌ Info: 
│   item_type_id = 1
└ @ Main C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\Bistum.jl:14


## Diözesen

In [15]:
csv_file = joinpath(data_path, "gs_dioceses.csv")
df_d = read_diocese(csv_file, :id_diocese);

┌ Info: Zeilen, Spalten: 
│   size(df_d) = (119, 16)
└ @ Main C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\Bistum.jl:36


## Bischofssitze
*2022-11-10* Bischofssitze neu zuordnen. Die bisherige Zuordnung nutzte die Namen der Orte, die freilich nicht eindeutig sind. Bei manchen Bischofssitzen ist die falsche GeoNames ID eingetragen (feature code "A" statt "P"). Das ist in der editierten Version korrigiert.

In [88]:
csv_file = joinpath(data_path, "gs_places_edit.csv")
df_p = read_place(csv_file);

┌ Info: Zeilen, Spalten: 
│   size(df_p) = (46921, 16)
└ @ Main C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\Bistum.jl:53


Ergänze `df_d` um `geonames_id`.

In [89]:
df_d_p = leftjoin(
    df_d, 
    select(df_p, [:id_places => :id_places, :place_name => :bs_name, :geonames_id => :geonames_id]), 
    on = :bishopric_seat => :id_places, 
    matchmissing = :notequal);

Lies Orte aus WIAG

In [90]:
table = "place";
sql = "select id, name, geonames_id from $(table)";
df_p_wiag = Wds.sql_df(sql);

Ergänze `df_d_p` um `place.id` aus WIAG

In [91]:
df_d_p = leftjoin(
    df_d_p,
    select(df_p_wiag, [:id => :bishopric_seat_id, :name => :wiag_bs_name, :geonames_id => :geonames_id]),
    on = :geonames_id,
    matchmissing = :notequal);

In [92]:
df_d_p[21:25, [:id_diocese, :diocese, :bs_name, :wiag_bs_name, :geonames_id, :bishopric_seat_id]]

Row,id_diocese,diocese,bs_name,wiag_bs_name,geonames_id,bishopric_seat_id
,String,String,String?,String?,Int64?,Int32?
1,14,Lübeck,Lübeck,Lübeck,2875601,386543
2,15,Magdeburg,Magdeburg,Magdeburg,2874545,387084
3,16,Mainz,Mainz,Mainz,2874225,387241
4,13,Lebus,Lebus,Lebus,2879795,388462
5,11,Köln,Köln,Köln,2886242,391607


Lies Bistümer aus WIAG

In [93]:
table = "diocese";
sql = "select id, name from $(table)";
df_d_wiag = Wds.sql_df(sql);

Ergänze `df_d_wiag` um einen neuen Wert für `bishopric_seat`

In [94]:
df_d_wiag = leftjoin(
    df_d_wiag,
    select(df_d_p, [:diocese, :bs_name, :wiag_bs_name, :geonames_id, :bishopric_seat_id]),
    on = :name => :diocese,
);

Keine Fehlermeldung: Alle Diözesen konnten zugeordnet werden

In [95]:
size(df_d_wiag)

(79, 6)

In [103]:
filter(:geonames_id => ismissing, df_d_wiag)

Row,id,name,bs_name,wiag_bs_name,geonames_id,bishopric_seat_id
,Int32,String?,String?,String?,Int64?,Int32?
1,55,Nordische Missionen,missing,missing,missing,missing


In [105]:
dropmissing!(df_d_wiag, :bs_name);

In [106]:
table = "diocese_update";
sql = "DROP TABLE IF EXISTS $(table)";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DROP TABLE IF EXISTS diocese_update", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [107]:
table = "diocese_update"
sql = "CREATE TEMPORARY TABLE $(table) " *
"(id INT, bishopric_seat_id INT) " *
"DEFAULT CHARSET=utf8";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "CREATE TEMPORARY TABLE diocese_update (id INT, bishopric_seat_id INT) DEFAULT CHARSET=utf8", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [108]:
table = "diocese_update"
Wds.filltable!(table, select(df_d_wiag, [:id, :bishopric_seat_id]))

┌ Info: Rows inserted: 78
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


78

In [109]:
sql = "UPDATE diocese, (SELECT id, bishopric_seat_id FROM diocese_update) as d_update " *
"SET diocese.bishopric_seat_id = d_update.bishopric_seat_id " *
"WHERE diocese.id = d_update.id";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "UPDATE diocese, (SELECT id, bishopric_seat_id FROM diocese_update) as d_update SET diocese.bishopric_seat_id = d_update.bishopric_seat_id WHERE diocese.id = d_update.id", 0, -1, 77, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)